<a href="https://colab.research.google.com/github/kocurvik/edu/blob/master/RO/materialy/cv10/walkrun_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

Using TensorFlow backend.


Uploadneme náš zip s datasetom.

In [5]:
from google.colab import files

uploaded = files.upload()

Saving data.zip to data.zip


In [6]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "data.zip" with length 63717725 bytes


Rozbalíme

In [7]:
!unzip data.zip

Archive:  data.zip
 extracting: test/run/run_0987572f.png  
  inflating: test/run/run_1073489f.png  
  inflating: test/run/run_13662896.png  
  inflating: test/run/run_3252a749.png  
  inflating: test/run/run_34713f03.png  
  inflating: test/run/run_7148ac65.png  
  inflating: test/run/run_7244bb3e.png  
  inflating: test/run/run_7666656e.png  
  inflating: test/run/run_8064cca3.png  
  inflating: test/run/run_849d4f97.png  
  inflating: test/run/run_8713cf1c.png  
  inflating: test/run/run_9417fa3e.png  
  inflating: test/run/run_94545263.png  
  inflating: test/run/run_a0564e69.png  
  inflating: test/run/run_acc06cbc.png  
  inflating: test/run/run_b0f5ef5d.png  
  inflating: test/run/run_b16636b7.png  
  inflating: test/run/run_b56d5e91.png  
  inflating: test/run/run_b5ff0667.png  
  inflating: test/run/run_bbbc236a.png  
  inflating: test/run/run_bde9d918.png  
  inflating: test/run/run_be2d5f70.png  
  inflating: test/run/run_c4e7d0ef.png  
  inflating: test/run/run_c9f91d3a.png

Vytvoríme objekty ktoré budú čítať naše dáta.

In [13]:
batch_size = 16

train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        'train',  # this is the target directory
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        'val',  # this is the target directory
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'test',  # this is the target directory
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

Found 600 images belonging to 2 classes.
Found 70 images belonging to 2 classes.
Found 71 images belonging to 2 classes.


Stiahneme si predtrénovaný model MobileNet V2 a nastavíme jeho vrstvy na netrénovanteľné.

In [9]:
pre_trained = MobileNetV2(input_shape=(224,224,3),  include_top = False)

print(pre_trained.summary())

for l in pre_trained.layers:
    l.trainable = False

Instructions for updating:
Colocations handled automatically by placer.
9412608/9406464 [==============================] - 0s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0]

Vytvoríme nový model, kde na začiatku bude naš predtrénovaný extraktor príznakov MobileNet. A potom zopár plne prepojených vrstiev.

In [10]:
model = Sequential()

model.add(pre_trained)
model.add(GlobalAveragePooling2D())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                40992     
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 2,299,521
Trainable params: 41,537
Non-trainable params: 2,257,984
_________________________________________________________________
None


Model natrénujeme a zistíme presnosť.

In [18]:
optimizer = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.fit_generator(train_generator,
        epochs=10,
        validation_data=validation_generator,
        steps_per_epoch=600,
        validation_steps=60)

model.evaluate_generator(test_generator)

Epoch 1/10


KeyboardInterrupt: ignored